In [13]:
# Zelle 1: Setup & Config
from google.cloud import bigquery
import pandas as pd
import logging

# Config
PROJECT_ID = "taxi-bi-project"  
DIM_DATASET = "dimensional"     # Quelle (Star Schema)
AGG_DATASET = "aggregational"       # Ziel (Data Marts)

client = bigquery.Client(project=PROJECT_ID)

# Hilfsfunktion, um Datasets zu finden/erstellen
def create_dataset_if_not_exists(dataset_id):
    full_dataset_id = f"{PROJECT_ID}.{dataset_id}"
    try:
        client.get_dataset(full_dataset_id)
        print(f"✅ Ziel-Dataset gefunden: {full_dataset_id}")
    except:
        print(f"Erstelle neues Dataset: {full_dataset_id} ...")
        # Wir holen uns die Region vom Quell-Dataset, damit alles gleich liegt (EU/US)
        src_ds = client.get_dataset(f"{PROJECT_ID}.{DIM_DATASET}")
        new_ds = bigquery.Dataset(full_dataset_id)
        new_ds.location = src_ds.location
        client.create_dataset(new_ds)
        print(f"✅ Dataset erstellt (Region: {src_ds.location})")

create_dataset_if_not_exists(AGG_DATASET)

✅ Ziel-Dataset gefunden: taxi-bi-project.aggregational


In [14]:
# Zelle 2: Aggregation 1 - Monthly KPIs (Management View)
# Diese Tabelle beantwortet: "Wie entwickeln sich Umsatz und Fahrtenzahlen?"

def create_monthly_kpis():
    print("--- 1. Erstelle Tabelle: agg_monthly_kpis ---")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_monthly_kpis`
    AS
    SELECT
        -- Dimensionen (Woran wollen wir schneiden?)
        d.year,
        d.month,
        d.month_name,
        d.quarter,
        f.source_system,      -- Yellow vs Green vs FHV
        v.vendor_name,        -- Creative Mobile vs Uber/Lyft Bases
        p.payment_description, -- Cash vs Credit
        
        -- Metriken (Hier wird gerechnet!)
        COUNT(f.trip_id) AS total_trips,
        
        -- Summen (Runden auf 2 Nachkommastellen spart Speicher und sieht besser aus)
        ROUND(SUM(f.total_amount), 2) AS total_revenue,
        ROUND(SUM(f.fare_amount), 2) AS total_fare,
        ROUND(SUM(f.tip_amount), 2) AS total_tips,
        
        -- Durchschnitte (KPIs)
        ROUND(AVG(f.total_amount), 2) AS avg_ticket_size,
        ROUND(AVG(f.trip_distance), 2) AS avg_distance_miles,
        ROUND(AVG(f.duration_minutes), 1) AS avg_duration_min

    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    
    -- JOINs zum Star Schema
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON f.pickup_date_key = d.date_key
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_vendor` v ON f.vendor_id = v.vendor_id
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_payment_type` p ON f.payment_type_id = p.payment_type_id

    -- FILTER:
    -- Wir wollen hier nur "echte" Fahrten für die Statistik.
    -- Wir schließen 0$-Fahrten (Geister) aus.
    -- Aber: Deine 3.80$ Fahrt (Short Trip) bleibt drin, weil total_amount > 0!
    WHERE f.total_amount > 0
    
    GROUP BY 1, 2, 3, 4, 5, 6, 7
    ORDER BY year DESC, month DESC, total_revenue DESC
    """
    
    try:
        job = client.query(sql)
        job.result() # Warten auf Fertigstellung
        print("✅ agg_monthly_kpis erfolgreich erstellt.")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_monthly_kpis()

--- 1. Erstelle Tabelle: agg_monthly_kpis ---
✅ agg_monthly_kpis erfolgreich erstellt.


In [15]:
def create_geo_stats_final():
    print("--- 2. Erstelle Tabelle: agg_geo_stats ---")
    
    # Wir nutzen SAFE_CAST und FORMAT_DATE, um sicherzugehen, dass der Key matcht
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_geo_stats` AS
    SELECT
        d.year,
        d.quarter,
        loc.borough AS pickup_borough,
        loc.zone AS pickup_zone,
        IFNULL(loc.service_zone, 'Other') AS service_zone,
        f.source_system,
        
        COUNT(f.trip_id) AS pickup_count,
        ROUND(SUM(f.total_amount), 0) AS total_revenue_generated,
        ROUND(AVG(f.tip_amount), 2) AS avg_tip_here

    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    
    -- JOIN zur Datumstabelle
    -- Wir wandeln das Trip-Datum in das Format YYYYMMDD um (Standard für date_key)
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d 
        ON CAST(FORMAT_DATE('%Y%m%d', DATE(f.pickup_datetime)) AS INT64) = CAST(d.date_key AS INT64)
        
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id

    WHERE f.total_amount > 0 
    GROUP BY 1, 2, 3, 4, 5, 6
    ORDER BY pickup_count DESC
    """
    
    try:
        client.query(sql).result()
        print("✅ agg_geo_stats erfolgreich erstellt!")
    except Exception as e:
        # Falls date_key doch ein DATE-Typ ist, w Fallback:
    
        fallback_sql = sql.replace(
            "CAST(FORMAT_DATE('%Y%m%d', DATE(f.pickup_datetime)) AS INT64) = CAST(d.date_key AS INT64)",
            "DATE(f.pickup_datetime) = d.date_key"
        )
        try:
            client.query(fallback_sql).result()
            print("✅ agg_geo_stats erfolgreich erstellt!")
        except Exception as e_inner:
            print(f"Kritischer Fehler: {e_inner}")

create_geo_stats_final()

--- 2. Erstelle Tabelle: agg_geo_stats ---
✅ agg_geo_stats erfolgreich erstellt!


In [23]:
# Zelle 4: Quality Check (Kurzer Blick auf das Ergebnis)
# Prüfen, ob die Tabellen gefüllt sind und die Geisterfahrten weg sind

def check_aggregation():
    print("\n--- CHECK: Monthly KPIs (Top 5 Rows) ---")
    query_kpi = f"""
    SELECT year, month_name, source_system, total_trips, total_revenue, avg_ticket_size 
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_monthly_kpis` 
    ORDER BY total_trips DESC 
    LIMIT 5
    """
    print(client.query(query_kpi).to_dataframe().to_string(index=False))

    print("\n--- CHECK: Geo Stats (Top 5 Zones) ---")
    query_geo = f"""
    SELECT pickup_borough, pickup_zone, source_system, pickup_count, total_revenue_generated
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_geo_stats`
    WHERE pickup_zone != 'NV' -- Wir ignorieren kurz die Unknowns
    ORDER BY pickup_count DESC
    LIMIT 5
    """
    print(client.query(query_geo).to_dataframe().to_string(index=False))

check_aggregation()


--- CHECK: Monthly KPIs (Top 5 Rows) ---


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


 year month_name source_system  total_trips  total_revenue  avg_ticket_size
 2010       June        YELLOW      4628917    48508294.19            10.48
 2011       June        YELLOW      4232780    44625418.33            10.54
 2010       June        YELLOW      4225814    43097295.83            10.20
 2011       June        YELLOW      4108120    44230599.26            10.77
 2015       June        YELLOW      4056209    73857571.43            18.21

--- CHECK: Geo Stats (Top 5 Zones) ---
pickup_borough           pickup_zone source_system  pickup_count  total_revenue_generated
     Manhattan Upper East Side South        YELLOW        541769                5080112.0
     Manhattan        Midtown Center        YELLOW        532814                5729221.0
     Manhattan Upper East Side South        YELLOW        530182                5210931.0
     Manhattan Upper East Side South        YELLOW        522302                5185967.0
     Manhattan        Midtown Center        YELLOW    

/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [24]:
def create_time_trends():
    print("--- 3. Erstelle Tabelle: agg_time_trends ---")
    
    sql = f"""
    CREATE OR REPLACE TABLE `taxi-bi-project.dimensional.dim_date` AS
    SELECT
        datum AS date_key,
        EXTRACT(YEAR FROM datum) AS year,
        EXTRACT(MONTH FROM datum) AS month,
        FORMAT_DATE('%B', datum) AS month_name,
        FORMAT_DATE('%A', datum) AS day_name,             -- WICHTIG: Für dein Skript
        EXTRACT(DAYOFWEEK FROM datum) AS day_of_week_num, -- WICHTIG: Für dein Skript
        EXTRACT(QUARTER FROM datum) AS quarter,
        CASE WHEN EXTRACT(DAYOFWEEK FROM datum) IN (1, 7) THEN TRUE ELSE FALSE END AS is_weekend
    FROM UNNEST(GENERATE_DATE_ARRAY('2010-01-01', '2025-12-31')) AS datum;
    """
    try:
        client.query(sql).result()
        print("✅ agg_time_trends erstellt.")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_time_trends()

--- 3. Erstelle Tabelle: agg_time_trends ---
✅ agg_time_trends erstellt.


In [25]:
def create_route_stats():
    print("--- 4. Erstelle Tabelle: agg_route_stats ---")
    
    # Hinweis: Ich habe LocationID zu location_id geändert. 
    # Bitte prüfe, ob die Spalte in deiner dim_location tatsächlich so heißt.
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_route_stats` AS
    SELECT
        d.year,
        d.month, -- Empfehlung: Monat hinzufügen für bessere Zeitreihen
        
        -- VON -> NACH
        pu.Borough AS pickup_borough,
        do.Borough AS dropoff_borough,
        
        COUNT(*) AS trip_count,
        ROUND(AVG(f.total_amount), 2) AS avg_cost,
        -- Falls duration_minutes in Fact_Trips existiert:
        ROUND(AVG(SAFE_DIVIDE(TIMESTAMP_DIFF(f.dropoff_datetime, f.pickup_datetime, SECOND), 60)), 1) AS avg_duration_min
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    -- Join auf deine neue dim_date (oder dim_datetime)
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON DATE(f.pickup_datetime) = d.date_key
    -- Achte hier auf den Spaltennamen: location_id vs LocationID
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` pu ON f.pickup_location_id = pu.location_id
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` do ON f.dropoff_location_id = do.location_id
    
    WHERE f.total_amount > 0 
      AND pu.Borough != 'Unknown' 
      AND do.Borough != 'Unknown'
      
    GROUP BY 1, 2, 3, 4
    ORDER BY trip_count DESC
    """
    try:
        client.query(sql).result()
        print("✅ agg_route_stats erfolgreich erstellt.")
    except Exception as e:
        # Detailliertere Fehlerausgabe
        print(f"❌ Fehler bei der Erstellung von agg_route_stats: {e}")

create_route_stats()

--- 4. Erstelle Tabelle: agg_route_stats ---
✅ agg_route_stats erfolgreich erstellt.


In [26]:
def create_airport_stats():
    print("--- 5. Erstelle Tabelle: agg_airport_trips ---")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_airport_trips` AS
    SELECT
        d.year,
        d.month_name,
        f.source_system,
        
        -- War es eine Fahrt ZUM oder VOM Flughafen?
        CASE 
            WHEN rc.rate_description LIKE '%JFK%' OR rc.rate_description LIKE '%Newark%' THEN 'Airport Rate'
            ELSE 'Standard Rate to Airport Zone'
        END AS trip_category,
        
        COUNT(*) AS total_trips,
        ROUND(AVG(f.total_amount), 2) AS avg_ticket,
        ROUND(AVG(f.tip_amount), 2) AS avg_tip
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON f.pickup_date_key = d.date_key
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_rate_code` rc ON f.rate_code_id = rc.rate_code_id
    
    -- Location Filter: 132=JFK, 138=LaGuardia, 1=Newark
    WHERE (f.pickup_location_id IN (132, 138, 1) OR f.dropoff_location_id IN (132, 138, 1))
      AND f.total_amount > 0
      
    GROUP BY 1, 2, 3, 4
    """
    try:
        client.query(sql).result()
        print("✅ agg_airport_trips erstellt.")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_airport_stats()

--- 5. Erstelle Tabelle: agg_airport_trips ---
✅ agg_airport_trips erstellt.


In [29]:
def create_quality_audit_mart():
    print("--- 8. Erstelle Tabelle: agg_quality_audit ---")
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_quality_audit` AS
    SELECT
        DATE_TRUNC(pickup_datetime, MONTH) as month,
        source_system,
        COUNT(*) as total_trips,
        COUNTIF(trip_distance = 0 AND source_system != 'FHV') as gps_failures,
        COUNTIF(pickup_location_id IN (263, 264)) as unknown_locations,
        COUNTIF(dq_issue_flag = TRUE) as total_issues
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips`
    GROUP BY 1, 2
    """
    client.query(sql).result()
    print("✅ agg_quality_audit erstellt.")
create_quality_audit_mart()

--- 8. Erstelle Tabelle: agg_quality_audit ---
✅ agg_quality_audit erstellt.


In [30]:
def create_shared_ride_stats():
    print("--- 7. Erstelle Tabelle: agg_shared_rides ---")
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_shared_rides` AS
    SELECT
        d.year,
        f.source_system,
        f.sr_flag,
        COUNT(*) AS trip_count,
        ROUND(AVG(f.fare_amount), 2) AS avg_fare -- Nur für Yellow/Green sinnvoll
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON f.pickup_date_key = d.date_key
    GROUP BY 1, 2, 3
    """
    client.query(sql).result()
    print("✅ agg_shared_rides erstellt.")
create_shared_ride_stats()

--- 7. Erstelle Tabelle: agg_shared_rides ---
✅ agg_shared_rides erstellt.


In [22]:
def check_new_marts():
    # Stelle sicher, dass die Variablen passen
    PROJECT_ID = "taxi-bi-project"
    AGG_DATASET = "aggregational"
    
    print("--- 📊 FINAL DATA MART CHECK ---")

    # 1. RUSH HOUR (Wann ist am meisten los?)
    print("\n🕐 TOP 5 ZEITFENSTER (agg_time_trends):")
    # Wir sortieren nach trip_count, um die geschäftigsten Stunden zu sehen
    sql_time = f"""
    SELECT day_name, hour_of_day, source_system, trip_count, avg_fare
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_time_trends`
    ORDER BY trip_count DESC
    LIMIT 5
    """
    try:
        df_time = client.query(sql_time).to_dataframe()
        print(df_time.to_string(index=False))
    except Exception as e:
        print(f"Fehler: {e}")

    # 2. ROUTEN (Wer fährt von wo nach wo?)
    print("\n📍 TOP 5 ROUTEN (agg_route_stats):")
    sql_routes = f"""
    SELECT pickup_borough, dropoff_borough, trip_count, avg_cost
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_route_stats`
    ORDER BY trip_count DESC
    LIMIT 5
    """
    try:
        df_routes = client.query(sql_routes).to_dataframe()
        print(df_routes.to_string(index=False))
    except Exception as e:
        print(f"Fehler: {e}")

    # 3. FLUGHAFEN (Die Cash Cows)
    print("\n✈️ FLUGHAFEN STATS (agg_airport_trips):")
    sql_air = f"""
    SELECT year, trip_category, source_system, total_trips, avg_ticket
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_airport_trips`
    ORDER BY total_trips DESC
    LIMIT 5
    """
    try:
        df_air = client.query(sql_air).to_dataframe()
        print(df_air.to_string(index=False))
    except Exception as e:
        print(f"Fehler: {e}")

check_new_marts()

--- 📊 FINAL DATA MART CHECK ---

🕐 TOP 5 ZEITFENSTER (agg_time_trends):


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


 day_name  hour_of_day source_system  trip_count  avg_fare
 Thursday           18        YELLOW      429644     29.42
Wednesday           18        YELLOW      423982     28.90
  Tuesday           18        YELLOW      403973     28.57
   Friday           18        YELLOW      397860     28.56
 Thursday           17        YELLOW      396463     31.39

📍 TOP 5 ROUTEN (agg_route_stats):


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


pickup_borough dropoff_borough  trip_count  avg_cost
     Manhattan       Manhattan    12410566     10.13
     Manhattan       Manhattan    12330416      9.96
     Manhattan       Manhattan    12316988      9.62
     Manhattan       Manhattan    11819999     11.80
     Manhattan       Manhattan    11183007     12.05

✈️ FLUGHAFEN STATS (agg_airport_trips):
 year                 trip_category source_system  total_trips  avg_ticket
 2014 Standard Rate to Airport Zone        YELLOW       614827       41.43
 2013 Standard Rate to Airport Zone        YELLOW       601767       40.22
 2012 Standard Rate to Airport Zone        YELLOW       596237       33.02
 2015 Standard Rate to Airport Zone        YELLOW       587991       43.20
 2011 Standard Rate to Airport Zone        YELLOW       578500       32.69


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
